In [1]:
import cv2
import albumentations as A
import numpy as np

In [2]:
import os
IMAGE_DIR = 'images'
LABEL_DIR = 'labels'
img_files = os.listdir(IMAGE_DIR)
label_files = os.listdir(LABEL_DIR)
img_path = os.path.join(IMAGE_DIR, f'{img_files[0]}')
label_path = os.path.join(LABEL_DIR, f'{label_files[0]}')

In [3]:
image = cv2.imread(img_path)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

In [4]:
bboxes = []

with open(label_path, 'r') as file:
  lines = file.readlines()

for line in lines:
  # bbox = []
  parts = line.strip().split()
  class_num = int(parts[0])
  bbox_coordinates = [float(part) for part in parts[1:]]
  # bbox = [class_num]+bbox_coordinates
  bboxes.append(bbox_coordinates)


In [5]:
transform = A.Compose(
  [
    A.Resize(width=640, height=480),
    A.RandomCrop(width=640, height=480),
    A.Rotate(limit=40, p=0.5, border_mode=cv2.BORDER_CONSTANT),
    A.HorizontalFlip(p=0.8),
    A.VerticalFlip(p=0.2),
    A.OneOf([
      A.Blur(blur_limit=5, p=0.5),
      A.MotionBlur(blur_limit=5, p=0.7)
    ], p=1.0)
  ], bbox_params=A.BboxParams(format="yolo",  min_visibility=0.3, label_fields=[])
)

In [6]:
images_list = [image]
saved_bboxes = [bboxes[0]]

In [7]:
for i in range(10):
  augmentations = transform(image=image, bboxes=bboxes)
  augmented_img = augmentations["image"]

  if len(augmentations["bboxes"]) == 0:
    continue

  images_list.append(augmented_img)
  saved_bboxes.append(augmentations["bboxes"])

In [17]:
RESULT_DIR = 'results'

In [21]:
for i, img in enumerate(images_list):
  result_path = os.path.join(RESULT_DIR, f'img-{i}.jpg')  # Add a valid file extension, such as '.jpg'
  img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
  cv2.imwrite(result_path, img)

In [22]:
for box in saved_bboxes:
  print(box)

[0.19323979591836743, 0.7252551020408161, 0.38647959183673486, 0.5341836734693876]
[(0.8067602040816326, 0.7252551020408161, 0.38647959183673486, 0.5341836734693877), (0.8040816326530611, 0.4926020408163266, 0.3918367346938777, 0.5198979591836735)]
[(0.7960188207060427, 0.5462785862035948, 0.40796235858791463, 0.6937800903774615), (0.7603030785173022, 0.3377490151727214, 0.47939384296539567, 0.6754980303454428)]
[(0.8067602040816326, 0.7252551020408161, 0.38647959183673486, 0.5341836734693877), (0.8040816326530611, 0.4926020408163266, 0.3918367346938777, 0.5198979591836735)]
[(0.8067602040816326, 0.7252551020408161, 0.38647959183673486, 0.5341836734693877), (0.8040816326530611, 0.4926020408163266, 0.3918367346938777, 0.5198979591836735)]
[(0.7968608103415857, 0.4429293589307115, 0.40627837931682864, 0.6874594443331872), (0.7629383000546793, 0.6575830525312253, 0.47412339989064134, 0.6768526524353546)]
[(0.7168650522516142, 0.760173324848407, 0.5119002885717748, 0.4796533503031859), (0.